In [2]:
import pandas as pd

job_description = pd.read_csv(r'C:\\Users\User\Desktop\Python Programming\NLP\group_project\NLP_project\job_descriptions.csv')
resumes = pd.read_csv(r'C:\Users\User\Desktop\Python Programming\NLP\group_project\NLP_project\resumes.csv')

In [7]:
print(resumes.columns)
print(job_description.columns)

Index(['ID', 'Professional_Summary', 'Work_Experience', 'Projects',
       'Certifications', 'Education', 'Skills', 'resume_text'],
      dtype='object')
Index(['ID', 'Company_Overview', 'Responsibilities', 'Required_Qualifications',
       'Preferred_Qualifications'],
      dtype='object')


## Keyword extraction
### TF-IDF

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Convert resume and job text to string format
resumes['resume_text'] = (
    resumes['Professional_Summary'].fillna('') + ' ' +
    resumes['Work_Experience'].fillna('') + ' ' +
    resumes['Projects'].fillna('') + ' ' +
    resumes['Certifications'].fillna('') + ' ' +
    resumes['Education'].fillna('') + ' ' +
    resumes['Skills'].fillna('')
)

job_description['job_description'] = (
    job_description['Company_Overview'].fillna('') + ' ' +
    job_description['Responsibilities'].fillna('') + ' ' +
    job_description['Required_Qualifications'].fillna('') + ' ' +
    job_description['Preferred_Qualifications'].fillna('')
)


resumes_clean = resumes['resume_text'].astype(str)
jobs_clean = job_description['job_description'].astype(str)

# TF-IDF Vectorizer
vectorizer = TfidfVectorizer(stop_words='english', max_features=100)
tfidf_resumes = vectorizer.fit_transform(resumes_clean)
resume_keywords = vectorizer.get_feature_names_out()

tfidf_jobs = vectorizer.fit_transform(jobs_clean)
job_keywords = vectorizer.get_feature_names_out()


In [14]:
import yake
kw_extractor = yake.KeywordExtractor(top=20)

# Convert resume and job text to string format
resumes['resume_text'] = (
    resumes['Professional_Summary'].fillna('') + ' ' +
    resumes['Work_Experience'].fillna('') + ' ' +
    resumes['Projects'].fillna('') + ' ' +
    resumes['Certifications'].fillna('') + ' ' +
    resumes['Education'].fillna('') + ' ' +
    resumes['Skills'].fillna('')
)

job_description['job_description'] = (
    job_description['Company_Overview'].fillna('') + ' ' +
    job_description['Responsibilities'].fillna('') + ' ' +
    job_description['Required_Qualifications'].fillna('') + ' ' +
    job_description['Preferred_Qualifications'].fillna('')
)

resumes_clean = resumes['resume_text'].astype(str)
jobs_clean = job_description['job_description'].astype(str)

# Apply keyword extraction
job_keywords_yake = jobs_clean.apply(lambda text: [kw for kw, _ in kw_extractor.extract_keywords(text)])
resume_keywords_yake = resumes_clean.apply(lambda text: [kw for kw, _ in kw_extractor.extract_keywords(text)])

def find_missing_keywords(job_kw, resume_kw):
    return list(set(job_kw) - set(resume_kw))

# Example for one-to-one comparison
for i in range(len(job_description)):
    job_kw = job_keywords_yake[i]
    resume_kw = resume_keywords_yake[i]
    missing = find_missing_keywords(job_kw, resume_kw)
    print(f"Missing in Resume {i} for Job {i}: {missing}")

Missing in Resume 0 for Job 0: ['Organic empowering architecture', 'Mentor junior', 'Finance company focused', 'marketing specialist solutions.', 'marketing specialist processes', 'Optimize existing digital', 'empowering architecture Monitor', 'Implement data-driven', 'marketing specialist', 'enhance digital marketing', 'improve digital marketing', 'Miller LLC', 'digital marketing', 'industry trends relevant', 'Stay updated', 'Mentor junior staff', 'Implement data-driven approaches', 'Optimize existing', 'existing digital marketing', 'digital marketing specialist']
Missing in Resume 1 for Job 1: ['reciprocal customer loyalty', 'speech therapist objectives', 'speech therapist processes', 'Monitor and improve', 'Implement data-driven', 'Lead cross-functional teams', 'speech therapist solutions.', 'customer loyalty Stay', 'enhance speech therapist', 'Centralized reciprocal customer', 'loyalty Stay updated', 'industry trends relevant', 'existing speech therapist', 'Lead cross-functional', 

In [16]:
output_df = pd.DataFrame({
    'job_id': job_description['ID'],
    'missing_keywords': [find_missing_keywords(job_keywords_yake[i], resume_keywords_yake[i]) for i in range(len(job_description))]
})
output_df.to_csv("missing_keywords_summary.csv", index=False)
